#### Requirements

In [23]:
import sys
import os

import geopandas as gpd
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.linear_model import LinearRegression

#### Constants

In [24]:
# General
BASE_PATH = os.path.dirname(os.getcwd())

# Mapping
CHANGE_TYPE_MAP = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}
CHANGE_STATUS_MAP = {'Greenland': 0, 'Land Cleared': 1, 'Excavation': 1, 'Materials Dumped': 3, 'Prior Construction': 3, 'Materials Introduced': 4, 'Construction Started': 5, 'Construction Midway': 6, 'Construction Done': 8, 'Operational': 10, None: None}

# Data
COLORS = ['red', 'green', 'blue']
METRICS = ['std', 'mean']
GEOGRAPHY_TYPES = ['Dense Forest', 'Grass Land', 'Sparse Forest', 'Farms', 'River',
                   'Coastal', 'Lakes', 'Barren Land', 'Desert', 'Hills', 'Snow'] 
URBAN_TYPES = ['Sparse Urban', 'Rural', 'Dense Urban', 'Urban Slum', 'Industrial']

# Columns groups
COLUMNS_TO_DROP = ['geography_type', 'urban_type']
DATE_COLUMNS = ['date0', 'date1', 'date2', 'date3', 'date4']
CHANGE_STATUS_COLUMNS = ['change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3', 'change_status_date4']
CHANGE_STATUS_VALUE_COLUMNS = ['change_status_value_date0', 'change_status_value_date1', 'change_status_value_date2', 'change_status_value_date3', 'change_status_value_date4']

# Output file
OUTPUT_FILE = 'preprocessed_train.geojson'

# Safety
MAPPED = False

#### Reading Data

In [25]:
## Read data
train_df = gpd.read_file(f'{BASE_PATH}/data/train.geojson', index_col=0)
#test_df = gpd.read_file(f'{BASE_PATH}/data/test.geojson', index_col=0)

#### Data preprocessing

In [26]:
## One-hot encoding
for geograph_type in GEOGRAPHY_TYPES:
    train_df[geograph_type] = train_df['geography_type'].apply(lambda x: 1 if geograph_type in x else 0)
for urban_type in URBAN_TYPES:
    train_df[urban_type] = train_df['urban_type'].apply(lambda x: 1 if urban_type in x else 0)

In [30]:
## Create new polygon features
train_df['area'] = train_df['geometry'].area
train_df['length'] = train_df['geometry'].length
train_df['area_to_length_ratio'] = train_df['area'] / train_df['length'] # the more, the closer to square
train_df['centroid_x'] = train_df['geometry'].centroid.x
train_df['centroid_y'] = train_df['geometry'].centroid.y
bounds = train_df['geometry'].bounds
train_df['angle'] = np.arctan((bounds['maxy']-bounds['miny'])/(bounds['maxx']-bounds['minx']))

/tmp/ipykernel_214543/3380409933.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_df['area'] = train_df['geometry'].area
/tmp/ipykernel_214543/3380409933.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_df['length'] = train_df['geometry'].length
/tmp/ipykernel_214543/3380409933.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_df['centroid_x'] = train_df['geometry'].centroid.x
/tmp/ipykernel_214543/3380409933.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometr

In [31]:
## Convert date from string to date
train_df[DATE_COLUMNS] = train_df[DATE_COLUMNS].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors='coerce'))

## Create deltas color[std, mean] 
for metric in METRICS:
    for color in COLORS:
        for i in range(2, 6):
            delta = train_df[f'img_{color}_{metric}_date{i}'] - train_df[f'img_{color}_{metric}_date{i-1}']
            train_df[f'img_{color}_{metric}_delta{i}'] = delta
        train_df[f'img_{color}_{metric}_delta_total'] = train_df[f'img_{color}_{metric}_date5'] - train_df[f'img_{color}_{metric}_date1']

## Create deltas time 
for i in range(1, 5):
    train_df[f'date_delta{i}'] = train_df[f'date{i}'] - train_df[f'date{i-1}']
train_df['date_delta_total'] = train_df[f'date4'] - train_df[f'date0']

In [32]:
## Standardizing colors mean by the proportion
for i in range(1, 6):
    color_sum = train_df[f'img_blue_mean_date{i}'] + train_df[f'img_green_mean_date{i}'] + train_df[f'img_red_mean_date{i}']
    for color in COLORS:
        train_df[f'img_{color}_mean_prop_date{i}'] = train_df[f'img_{color}_mean_date{i}']/color_sum

## Create img_{color}_mean_prop_rate
num_samples = train_df.shape[0]
ones = np.ones((num_samples,5,1))

for color in COLORS:
    coef = np.zeros((num_samples))
    COLOR_MEAN_COLUMNS = [f'img_{color}_mean_prop_date{i}' for i in range (1,6)]
    
    Y = np.array(train_df[COLOR_MEAN_COLUMNS].astype(float))
    X = np.array(train_df[DATE_COLUMNS].astype(int))[:,:,np.newaxis]
    X = np.dstack((ones,X))
    nan_mask = np.isnan(Y) | np.isnan(X[:,:,1])
    X[nan_mask,:] = 0
    Y[nan_mask] = 0

    eye = np.eye(2)*0.0001
    for i in range(num_samples):
        x = X[i].reshape((5,2))
        y = Y[i].reshape((5))
        coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]
        
    train_df[f'img_{color}_mean_prop_rate'] = coef
    
## Create img_{color}_std_rate
for color in COLORS:
    coef = np.zeros((num_samples))
    COLOR_STD_COLUMNS = [f'img_{color}_std_date{i}' for i in range (1,6)]
    
    Y = np.array(train_df[COLOR_STD_COLUMNS].astype(float))
    X = np.array(train_df[DATE_COLUMNS].astype(int))[:,:,np.newaxis]
    X = np.dstack((ones,X))
    nan_mask = np.isnan(Y) | np.isnan(X[:,:,1])
    X[nan_mask,:] = 0
    Y[nan_mask] = 0

    eye = np.eye(2)*0.0001
    for i in range(num_samples):
        x = X[i].reshape((5,2))
        y = Y[i].reshape((5))
        coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]
        
    train_df[f'img_{color}_std_rate'] = coef

In [33]:
## Map change_type
if ~MAPPED:
    train_df['change_type'] = train_df['change_type'].map(CHANGE_TYPE_MAP)
    MAPPED = True

In [34]:
## Create change_status_values
train_df[CHANGE_STATUS_VALUE_COLUMNS] = train_df[CHANGE_STATUS_COLUMNS].replace(CHANGE_STATUS_MAP)

## Create civilization_rate
num_samples = train_df.shape[0]
coef = np.zeros((num_samples))
time_ctt = 1e9*60*90*24
ones = np.ones((num_samples,5,1))

Y = np.array(train_df[CHANGE_STATUS_VALUE_COLUMNS].astype(float))
Y_nan_mask = np.isnan(Y)
X = np.array(train_df[DATE_COLUMNS].astype(int))[:,:,np.newaxis]/time_ctt
X = np.dstack((ones,X))
X[Y_nan_mask,:] = 0
Y[Y_nan_mask] = 0

eye = np.eye(2)*0.0001
for i in range(num_samples):
    x = X[i].reshape((5,2))
    y = Y[i].reshape((5))
    coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]
    #print(y, train_df["change_type"].iloc[i])
train_df["civilizating_rate"] = coef


/tmp/ipykernel_214543/3193938916.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df[CHANGE_STATUS_VALUE_COLUMNS] = train_df[CHANGE_STATUS_COLUMNS].replace(CHANGE_STATUS_MAP)


#### Finalization and Save

In [35]:
## Drop uncessary columns
train_df = train_df.drop(columns=COLUMNS_TO_DROP)
train_df = train_df.drop(columns=CHANGE_STATUS_COLUMNS)

In [37]:
## ANOVA for each feature
from scipy.stats import f_oneway

# Assuming 'change_type' is the column name for classification effects
change_type_column = 'change_type'

# Group the data by 'change_type'
grouped_data = train_df.groupby(change_type_column)

# List to store F-values
f_values = []

# Loop through each feature (column) and calculate F-value
for feature_column in train_df.columns:
    if feature_column != change_type_column and feature_column != 'geometry':  # Skip the 'change_type' column itself and 'geometry'
        feature_values = [group[feature_column].dropna() for _, group in grouped_data]
        f_statistic, p_value = f_oneway(*feature_values)
        f_values.append((feature_column, f_statistic, p_value))

# Convert to DataFrame for easier analysis
f_values_df = pd.DataFrame(f_values, columns=['Feature', 'F-value', 'P-value'])

In [38]:
# Print or analyze the F-values DataFrame
sorted_f_values_df= f_values_df.sort_values(by='F-value')
sorted_f_values_df.to_csv("ANOVA.csv", sep='\t', index=False)

In [39]:
## To inspect
#train_df.head()

In [40]:
## Save output file
train_df.to_file(OUTPUT_FILE) 

ValueError: Invalid field type <class 'pandas._libs.tslibs.timedeltas.Timedelta'>